# Regresión Lineal Multivariable

En este notebook se va a desarrollar un ejemplo de Regresión lineal multivariable.
Al igual que en los ejercicios anteriores, debes completar el codigo en las celdas donde diga "implementa tu codigo aqui"

In [1]:
%matplotlib inline


import pandas as pd
from sklearn import linear_model
import data_helper
import numpy as np
import matplotlib.pyplot as plt

In [2]:
 pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Cargando los datos

In [5]:
data_casas = pd.read_csv('C:\\Users\\User\\Desktop\\Universidad\\Machine Learning\\Ejercicio_clase15\\housing_data.csv')

In [6]:
data_casas.head(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,nan,SS,Jellis,3/09/2016,2.500,3067.000,...,1.000,1.000,126.000,nan,nan,Yarra City Council,-37.801,144.996,Northern Metropolitan,4019.000
1,Abbotsford,85 Turner St,2,h,1480000.000,S,Biggin,3/12/2016,2.500,3067.000,...,1.000,1.000,202.000,nan,nan,Yarra City Council,-37.800,144.998,Northern Metropolitan,4019.000
2,Abbotsford,25 Bloomburg St,2,h,1035000.000,S,Biggin,4/02/2016,2.500,3067.000,...,1.000,0.000,156.000,79.000,1900.000,Yarra City Council,-37.808,144.993,Northern Metropolitan,4019.000
3,Abbotsford,18/659 Victoria St,3,u,nan,VB,Rounds,4/02/2016,2.500,3067.000,...,2.000,1.000,0.000,nan,nan,Yarra City Council,-37.811,145.012,Northern Metropolitan,4019.000
4,Abbotsford,5 Charles St,3,h,1465000.000,SP,Biggin,4/03/2017,2.500,3067.000,...,2.000,0.000,134.000,150.000,1900.000,Yarra City Council,-37.809,144.994,Northern Metropolitan,4019.000
5,Abbotsford,40 Federation La,3,h,850000.000,PI,Biggin,4/03/2017,2.500,3067.000,...,2.000,1.000,94.000,nan,nan,Yarra City Council,-37.797,144.997,Northern Metropolitan,4019.000
6,Abbotsford,55a Park St,4,h,1600000.000,VB,Nelson,4/06/2016,2.500,3067.000,...,1.000,2.000,120.000,142.000,2014.000,Yarra City Council,-37.807,144.994,Northern Metropolitan,4019.000
7,Abbotsford,16 Maugie St,4,h,nan,SN,Nelson,6/08/2016,2.500,3067.000,...,2.000,2.000,400.000,220.000,2006.000,Yarra City Council,-37.797,144.996,Northern Metropolitan,4019.000
8,Abbotsford,53 Turner St,2,h,nan,S,Biggin,6/08/2016,2.500,3067.000,...,1.000,2.000,201.000,nan,1900.000,Yarra City Council,-37.800,144.997,Northern Metropolitan,4019.000
9,Abbotsford,99 Turner St,2,h,nan,S,Collins,6/08/2016,2.500,3067.000,...,2.000,1.000,202.000,nan,1900.000,Yarra City Council,-37.800,144.999,Northern Metropolitan,4019.000


In [7]:
data_casas.shape

(31420, 21)

## Feature Engineering

Para tener una mejor idea de que features nos pueden servir, generamos una matriz de correlación. 

En esta sección vamos a remover la columna de vendedor ya que no va a ser útil para el análisis y crear una nueva feature que se llame **numero_habitaciones** que va a ser la suma de las features **dormitorios** y **banios**

In [9]:
corr_matrix = data_casas.corr()
corr_matrix["Price"].sort_values(ascending=False)

Price            1.000
Rooms            0.468
Bedroom2         0.438
Bathroom         0.437
Car              0.211
Longtitude       0.195
BuildingArea     0.100
Postcode         0.048
Landsize         0.034
Propertycount   -0.057
Distance        -0.203
Lattitude       -0.205
YearBuilt       -0.327
Name: Price, dtype: float64

#### Removiendo la columna 'Vendedor'

Para eliminar una columna en un data frame de pandas utilizamos:
```
DataFrame.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')
```
Si seteamos el valor de axis a 1, indicamos que queremos eliminar una columna. Ejemplo

```
+-----------------+
|A  | B |  C |  D |
+-----------------+   
| 0 | 1 |  2 |  3 |
| 4 | 5 |  6 |  7 |
| 8 | 9 | 10 | 11 |
+-----------------+   
```

```
df.drop(['B', 'C'], axis=1)
```

```
+--------+
|A  |  D |
+--------+   
| 0 |  3 |
| 4 |  7 |
| 8 | 11 |
+--------+ 
```

In [14]:
data_casas = data_casas.drop(['SellerG'], axis=1) 

In [17]:
# Mostramos las primeras 5 filas para validar que eliminamos la columna 'Vendedor'
data_casas.head(5)

,Suburb,Address,Rooms,Type,Price,Method,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,nan,SS,3/09/2016,2.500,3067.000,2.000,1.000,1.000,126.000,nan,nan,Yarra City Council,-37.801,144.996,Northern Metropolitan,4019.000
1,Abbotsford,85 Turner St,2,h,1480000.000,S,3/12/2016,2.500,3067.000,2.000,1.000,1.000,202.000,nan,nan,Yarra City Council,-37.800,144.998,Northern Metropolitan,4019.000
2,Abbotsford,25 Bloomburg St,2,h,1035000.000,S,4/02/2016,2.500,3067.000,2.000,1.000,0.000,156.000,79.000,1900.000,Yarra City Council,-37.808,144.993,Northern Metropolitan,4019.000
3,Abbotsford,18/659 Victoria St,3,u,nan,VB,4/02/2016,2.500,3067.000,3.000,2.000,1.000,0.000,nan,nan,Yarra City Council,-37.811,145.012,Northern Metropolitan,4019.000
4,Abbotsford,5 Charles St,3,h,1465000.000,SP,4/03/2017,2.500,3067.000,3.000,2.000,0.000,134.000,150.000,1900.000,Yarra City Council,-37.809,144.994,Northern Metropolitan,4019.000


#### Creando la feature **numero_habitaciones**

Ejemplo:
```
+--------+
|A  |  D |
+--------+   
| 0 |  3 |
| 4 |  7 |
| 8 | 11 |
+--------+ 
```

```
df['nueva_columna'] = df['A'] * df['C']
```

```
+-----------------------+
|A  |  D | nueva_columna|
+-----------------------+   
| 0 |  3 |            0 |
| 4 |  7 |           28 |
| 8 | 11 |           88 |
+-----------------------+ 
```

In [18]:
data_casas['numero_habitaciones'] = data_casas['Rooms'] * data_casas['Bedroom2']



In [19]:
# Validamos que nuestro data frame tenga la nueva columna 
data_casas.head(5)

,Suburb,Address,Rooms,Type,Price,Method,Date,Distance,Postcode,Bedroom2,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,numero_habitaciones
0,Abbotsford,68 Studley St,2,h,nan,SS,3/09/2016,2.500,3067.000,2.000,...,1.000,126.000,nan,nan,Yarra City Council,-37.801,144.996,Northern Metropolitan,4019.000,4.000
1,Abbotsford,85 Turner St,2,h,1480000.000,S,3/12/2016,2.500,3067.000,2.000,...,1.000,202.000,nan,nan,Yarra City Council,-37.800,144.998,Northern Metropolitan,4019.000,4.000
2,Abbotsford,25 Bloomburg St,2,h,1035000.000,S,4/02/2016,2.500,3067.000,2.000,...,0.000,156.000,79.000,1900.000,Yarra City Council,-37.808,144.993,Northern Metropolitan,4019.000,4.000
3,Abbotsford,18/659 Victoria St,3,u,nan,VB,4/02/2016,2.500,3067.000,3.000,...,1.000,0.000,nan,nan,Yarra City Council,-37.811,145.012,Northern Metropolitan,4019.000,9.000
4,Abbotsford,5 Charles St,3,h,1465000.000,SP,4/03/2017,2.500,3067.000,3.000,...,0.000,134.000,150.000,1900.000,Yarra City Council,-37.809,144.994,Northern Metropolitan,4019.000,9.000


#### Removiendo las columnas  'dormitorios' y 'banios"

para nuestro modelo ya no vamos a utilizar las features **dormitorios** y **banios**, en lugar de estas vamos a utilizar la nueva feature **numero_habitaciones**

In [20]:
data_casas = data_casas.drop(['Rooms', 'Bathroom'], axis=1) 

In [21]:
data_casas.head(5)

,Suburb,Address,Type,Price,Method,Date,Distance,Postcode,Bedroom2,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,numero_habitaciones
0,Abbotsford,68 Studley St,h,nan,SS,3/09/2016,2.500,3067.000,2.000,1.000,126.000,nan,nan,Yarra City Council,-37.801,144.996,Northern Metropolitan,4019.000,4.000
1,Abbotsford,85 Turner St,h,1480000.000,S,3/12/2016,2.500,3067.000,2.000,1.000,202.000,nan,nan,Yarra City Council,-37.800,144.998,Northern Metropolitan,4019.000,4.000
2,Abbotsford,25 Bloomburg St,h,1035000.000,S,4/02/2016,2.500,3067.000,2.000,0.000,156.000,79.000,1900.000,Yarra City Council,-37.808,144.993,Northern Metropolitan,4019.000,4.000
3,Abbotsford,18/659 Victoria St,u,nan,VB,4/02/2016,2.500,3067.000,3.000,1.000,0.000,nan,nan,Yarra City Council,-37.811,145.012,Northern Metropolitan,4019.000,9.000
4,Abbotsford,5 Charles St,h,1465000.000,SP,4/03/2017,2.500,3067.000,3.000,0.000,134.000,150.000,1900.000,Yarra City Council,-37.809,144.994,Northern Metropolitan,4019.000,9.000


## Hipótesis en RLMV: 

En esta funcion vamos a obtener los resultados de la hipotesis

$h_{\theta}=\theta _{0}+\theta _{1}X_{1}+\theta _{2}X_{2}+\ldots +\theta _{n}X_{n}$

**Features:** data frame con las features que se utilizaran para predecir la hipotesis

**parametros:** vector con los valores theta del modelo

In [13]:
def get_hipotesis(features, theta_values):
    ## Calculo de la hipotesis de froma vectorizada ##
       
    y = X.dot(theta_values).flatten()
    
    return y

In [30]:
##### EJEMPLO #######

# Obetenemos la Features que vamos a utilizar en nuesto modelo
features_x = data_casas[['Area_Lote', 'numero_habitaciones', 'paqueos', 'anio_construccion']]

# Recordemos que x0=1, por lo que agreamos una features de valores 1
features_x['interceptor'] = 1

# Creando la Matriz X de features
X = np.array(features_x)

theta_values = [992.6628870692274, 254599.25629753844, 55288.30299300452, -5152.457568534031, 9937364.817429375]

y_predicted = get_hipotesis(X, theta_values)

KeyError: "['Area_Lote' 'paqueos' 'anio_construccion'] not in index"

## Costo en RLMV: 
$h_{\theta}=\theta _{0}+\theta _{1}X_{1}+\theta _{2}X_{2}+\ldots +\theta _{n}X_{n}$


$J(\theta _{0},\theta _{1},\theta _{2},\ldots,\theta _{n}) ={\frac {1}{2m}}\sum _{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})^{2}$

In [37]:
def cost_function(x, y, theta_values):
    """
    cost_function(X, y, theta_values) computes the cost of using beta as the
    parameter for linear regression to fit the data points in X and y
    """
    ## number of training examples
    m = len(y)

    ## Calcula los valores de la hipotesis utilizando la funcion
    h = get_hipotesis(x, theta_values)
    
    J= np.sum((h*y)**2)/2*m
    return J

In [38]:
### EJECUTA ESTA CELDA PARA VALIDAR TU FUNCION DE COSTO ######

features_x = data_casas[['Area_Lote', 'numero_habitaciones', 'paqueos', 'anio_construccion']]
y = data_casas['Price']
theta_values = [992.6628870692274, 254599.25629753844, 55288.30299300452, -5152.457568534031, 9937364.817429375]

cost_function(features_x, y, theta_values)

KeyError: "['Area_Lote' 'paqueos' 'anio_construccion'] not in index"

El resultado de la celda anterior deberia de ser 127938086592.94948

## Proceso de entrenamiento/aprendizaje en RLMV: 

Repetir{

$\theta_{j} := \theta _{j}- \alpha {\frac {\partial}{\partial \theta_{j}}}J(\theta _{0},\theta _{1},\theta _{2},\ldots,\theta _{n})$

}
        
Simultaneamente para cada $j = 0,\ldots,n $


### Gradient descent en RLMV: 
Repetir{

$\theta_{j} := \theta _{j} - \alpha {\frac {1}{m}} \sum _{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{j}$

}
        
Simultaneamente para cada $j = 0,\ldots,n $

#### Ejemplo en codigo
Esto lo vamos a implementar en un ciclo for, por ejemplo 

```
for i, theta in enumerate(theta_vector):
  nuevo_theta = <tu implementacion de la formula aca>
  
  ### aqui agregamos el nuevo valor the theta[i] a un vector temporal
  theta_vector_temp.append(nuevo_theta)
  
## Cuando finaliza el ciclo actualizamos el vector theta con los nuevos valores
theta_vector = theta_vector_temp

```

El resultado de este ciclo seria el siguiente, para el caso de una regresion con 2 variables

$\theta_{0} := \theta _{j} - \alpha {\frac {1}{m}} \sum _{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{0}$

$\theta_{1} := \theta _{1} - \alpha {\frac {1}{m}} \sum _{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{1}$

$\theta_{2} := \theta _{2} - \alpha {\frac {1}{m}} \sum _{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{2}$

Al implementar esta formula en Python, adentro de la sumatoria $(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{j}$
El codigo para multiplicar por $x^{(i)}_{j}$  seria :
```* X[:,i]```

In [1]:
######### FEATURES ###############################
# Obetenemos la Features que vamos a utilizar en nuesto modelo
features_x = data_casas[['Area_Lote', 'numero_habitaciones', 'paqueos', 'anio_construccion']]
# Recordemos que x0=1, por lo que agreamos una features de valores 1
features_x['interceptor'] = 1
# Creando la Matriz X de features
X = np.array(features_x)

###### Valores de y ############    
y = data_casas['Precio']

## Numero de feautres ##
number_features = 4 + 1

## Creamos un vector donde se almacenan los valores de Theta,
## lo inicializamos con numeros aleatorios
theta_vector = np.random.rand(number_features)

## Numero de iteracions ##
iterations = 200

## Alpha ##
alpha = 0.0000005

m = len(y) 

## loss vector
loss_vec = []
## Cost vector
cost_vect = []


for iteration in range(iterations):
      def cost_function(x, y, theta_values):
             m = len(y)
    hipotesis = get_hipotesis(X, theta_vector)
    
    
    theta_vector_temp = []
    for i, theta in enumerate(theta_vector):
     
        nuevo_theta = nuevo_theta = theta_vector-alpha*(1/(int(m))*np.sum(np.subtract(lm.predict(features),y)*X[:,i]))

        
        ### aqui agregamos el nuevo valor the theta[i] a un vector temporal
        theta_vector_temp.append(nuevo_theta)
    
    
    ## Actualizamos el vector theta con los nuevos valores
    theta_vector = theta_vector_temp
    
      cost = cost_function(x, y, theta_values)
    
    ## Guardando el valor del costo para graficarlo ##
    cost_vect.append(cost)
    
    ## Este codigo no hay que modificarlo ##
    if(iteration % 20 == 0):
        print('#####################')
        print('Iteracion: ', iteration)
        print('Costo: ', cost)
        

print('')        
print('Costo: ', cost)



plt.plot(cost_vect, color='red', label = 'Costo')
plt.show()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 36)

### Costo en cada iteracion

Al ejecutar tu implementacion de Gradient descent en RLMV, el costo debe disminuir en cada iteracion 
por lo que la grafica de costo resultante es la siguiente:


<img src="images/cost_over_time.png" width="400">

In [ ]:
data_casas['Prediccion'] = get_hipotesis(X, theta_vector)
data_casas = data_casas[['Area_Lote', 'numero_habitaciones', 'paqueos', 'anio_construccion', 'Precio', 'Prediccion']]
data_casas.head(20)

# RLMV con Scikit Learn

In [9]:
features = data_casas[['Area_Lote', 'dormitorios', 'paqueos', 'anio_construccion']]
y = data_casas['Precio']

In [10]:
lm = linear_model.LinearRegression()
lm.fit(features,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

#### Modelo

In [12]:
t1 = lm.coef_[0]
t2 = lm.coef_[1]
t3 = lm.coef_[2]
t4 = lm.coef_[3]
b = lm.intercept_
print(' y = {0}x + {1}x + {2}x  {3}x + {4}'.format(t1,t2,t3,t4, b))

 y = 1220.405076131394x + 359246.35711640434x + 67915.23361095643x  -4022.9490705421395x + 7770894.079227889


In [14]:
data_casas['Prediccion'] = lm.predict(features)
data_casas = data_casas[['Area_Lote', 'dormitorios', 'paqueos', 'anio_construccion', 'Precio', 'Prediccion']]

In [15]:
data_casas.head(10)

,Area_Lote,dormitorios,paqueos,anio_construccion,Precio,Prediccion
2,79.000,2,0,1900,1035000.000,942195.560
4,150.000,3,0,1900,1465000.000,1388090.678
6,142.000,4,2,2014,1600000.000,1414788.068
11,210.000,3,0,1910,1876000.000,1421085.492
14,107.000,2,2,1890,1636000.000,1152426.861
18,75.000,2,2,1900,1097000.000,1073144.407
24,190.000,3,2,2005,1350000.000,1150327.696
25,94.000,2,1,2009,750000.000,589915.422
30,97.000,2,2,1890,1310000.000,1140222.810
32,110.000,3,1,1880,1200000.000,1487648.690
